![](./images/header.png)

# Fetch satellite data using OpenEO

`OpenEO` is a platform and a python library that allows to gather satellite data from extensive catalogues, and preprocess the data before being downloaded.

No mosaicing is required when using OpenEO, as the platform will do it for the user automatically. In this notebook, OpenEO will be used to fetch data in order to perfrom later crop type mapping.

In [ ]:
# Load the data
import openeo
from shapely.geometry import mapping
from pathlib import Path

out_path = Path('/data/users/Public/couchard/S2_L2A_Malawi.nc')

if not out_path.exists():
    spatial_extent = mapping(box(west, south, east, north))
    spatial_extent['crs'] = 'EPSG:4326'
    temporal_extent = ['2021-10-01', '2022-08-01']
    bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12', 'SCL']
    connection = openeo.connect('http://openeo.vito.be').authenticate_oidc()

    # Load the S2 collection
    cube = connection.load_collection(
        'SENTINEL2_L2A',
        spatial_extent=spatial_extent,
        temporal_extent=temporal_extent,
        bands=bands
    )

    job_options = {
        'sentinel-hub': {
            'client-alias': 'ai4food'
        },
    }

    job = cube.create_job(
        out_format='NetCDF', title='Malawi S2 extraction', job_options=job_options
    )

    job.start_and_wait()

    results = job.get_results()
    for asset in results.get_assets():
        if asset.metadata['type'].startswith('application/x-netcdf'):
            asset.download(out_path)

In [ ]:
# Once the data is downloaded, we change the metdata to be common around all bands
import rioxarray

out_path_10m = '/data/users/Public/couchard/S2_L2A_Malawi_10m.nc'
out_path_20m = '/data/users/Public/couchard/S2_L2A_Malawi_20m.nc'

# Split the data in 20m and 10m
inds = rioxarray.open_rasterio(out_path)
inds.attrs.update({'bounds': inds.rio.bounds()})

inds = inds.rename({'t': 'timestamp'})

inds_20m = inds[['B05', 'B06', 'B07', 'B11', 'B12']]
inds_20m.attrs.update({'resolution': 20.0})
inds_20m['SCENECLASSIFICATION'] = inds['SCL']

inds_10m = inds[['B02', 'B03', 'B04', 'B08']]
inds_10m.attrs.update({'resolution': 10.0})

inds_10m.to_netcdf(out_path_10m)
inds_20m.to_netcdf(out_path_20m)